<a href="https://colab.research.google.com/github/Kuol287/Kuol287/blob/main/SUICIDE_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
import string
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


**Mounting the Drive to Access Data**

In [2]:
Data_path="/content/drive/MyDrive/Graduation Projectt/DATASET/Suicide_Detection.csv"
df = pd.read_csv(Data_path)

**Display the first few dat rows**

In [3]:
print(df.head())

   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  I’m so lostHello, my name is Adam (16) and I’v...      suicide


**Display the data summary of our data**

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  232074 non-null  int64 
 1   text        232074 non-null  object
 2   class       232074 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.3+ MB
None


**Ensure that columns and Labels are correctly named**

In [5]:
text_column = 'text'  # Change this if your column name differs
label_column = 'label'  # Change this if your column name differs

**Text Preprocessing**

In [6]:
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-letter characters
    tokens = word_tokenize(text)  # Tokenization

    if not tokens:
        return ''

    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopword removal

    if not tokens:
        return ''

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return ' '.join(tokens)

**Apply Preprocessing**

In [7]:
df[text_column] = df[text_column].astype(str).apply(preprocess_text)

**Before vectorization, remove rows with empty text**

In [8]:
df = df[df[text_column] != '']

**Reset index after removing rows**

In [9]:
df.reset_index(drop=True, inplace=True)

** Convert categories to binary (1 for suicide, 0 for non-suicide)**

In [10]:
label_column = 'class'  # Change this to match your dataset's column name
df[label_column] = df[label_column].map({'suicide': 1, 'non-suicide': 0})

<ipython-input-10-c588bd2fe556>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label_column] = df[label_column].map({'suicide': 1, 'non-suicide': 0})


**Display to check the first entries to Confirm that  0 and 1 are used**

In [11]:
print(df.head())

   Unnamed: 0                                               text  class
0           2  ex wife threatening suiciderecently left wife ...      1
1           3  weird dont get affected compliment coming some...      0
2           4  finally almost never hear bad year ever swear ...      0
3           8                     need helpjust help im cry hard      1
4           9  im losthello name adam ive struggling year im ...      1


**Convert text to numerical representation**

In [12]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df[text_column])
y = df[label_column]

**Handle NaN values in the target variable**

In [13]:
df.dropna(subset=[label_column], inplace=True)

**Reset index after dropping rows**

In [14]:
df.reset_index(drop=True, inplace=True)

**Update X and y after removing NaN values**

In [15]:
X = vectorizer.fit_transform(df[text_column])
y = df[label_column]

**Split dataset**

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Train SVM Using SGD**

In [17]:
sgd_svm = SGDClassifier(loss="hinge",  # Hinge loss for SVM
                        penalty="l2",  # L2 regularization
                        alpha=0.0001,  # Regularization strength
                        max_iter=1000,  # Number of iterations
                        tol=1e-3,  # Stopping criteria
                        random_state=42)

sgd_svm.fit(X_train, y_train)


SGDClassifier(random_state=42)

**Train SVM model**

In [18]:

#svm_model = SVC(kernel='linear', probability=True)
#svm_model.fit(X_train, y_train)

**Train the Naive Bayes Model**

In [22]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

**Evaluate the Models**

In [29]:
accuracy = accuracy_score(y_test, y_pred)
print(f"SGD SVM Accuracy: {accuracy:.4f}")
y_pred = sgd_svm.predict(X_test)
y_pred_nb = nb_model.predict(X_test)
#print("SVM Model Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Naïve Bayes Model Accuracy:", accuracy_score(y_test, y_pred_nb))
#print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))
print("Naïve Bayes Classification Report:\n", classification_report(y_test, y_pred_nb))


SGD SVM Accuracy: 0.9279
Naïve Bayes Model Accuracy: 0.9023039289639863
Naïve Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.86      0.90     23203
           1       0.87      0.95      0.91     23196

    accuracy                           0.90     46399
   macro avg       0.91      0.90      0.90     46399
weighted avg       0.91      0.90      0.90     46399



**Saving the Models**

In [28]:
with open('sgd_svm.pkl', 'wb') as f:
    pickle.dump(sgd_svm, f)
with open('nb_model.pkl', 'wb') as f:
    pickle.dump(nb_model, f)
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Models and vectorizer saved successfully.")

Models and vectorizer saved successfully.
